<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=26fb8c32a4051abfefd8ff269093fbd49e177d50a831f2a5cb3170d94e1ab115
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-17 10:19:49
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.41 C
-------------------
Today PnL: 15.51 K (0.11%)
Current PnL: -20.98 L (-13.98%)
CY Booked + Current PnL: -9.08 L (-6.05%)
-------------------
Total profit:  1.87 L
Total loss:  -22.85 L
-------------------
Total Booked + Current PnL: 17.90 L (14.54%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.47%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.14 L (62.73%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 7.85%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.44,58.0,X-LC,3.88,214748.0,10248.0,13057.0,-0.12,5.01,6.08,11.39,10.0,0.78,1.54,14.41,X40N,ATH,PHARMA
83,VOLTAS,1530.00,2.14,61.0,X-MC,1.35,214350.0,22608.0,15155.0,0.95,11.79,7.07,19.69,99.0,1.49,1.54,19.39,XY25,NTT,AC
76,TTKPRESTIG,770.00,97.65,52.0,M-SC,1.63,85124.0,-15653.0,15748.0,-0.66,-15.53,18.50,0.09,245.0,-0.99,0.61,11.83,OX40N,NTT,DURABLES
33,ICICIGI,2260.25,-14.07,64.0,X-MC,2.32,147919.0,11786.0,19333.0,-0.56,8.66,13.07,22.86,91.0,0.61,1.06,22.36,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-1.60,74.0,X-LC,2.89,302726.0,37300.0,19496.0,1.34,14.05,6.44,21.40,11.0,1.91,2.17,21.63,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.00,-41.17,64.0,M-SC,4.30,99510.0,8012.0,70742.0,7.19,8.76,71.09,86.07,223.0,0.11,0.71,44.51,XR,NTT,DURABLES
5,ASIANPAINT,3465.66,-11.88,66.0,X-LC,5.31,222385.0,-29383.0,82594.0,4.87,-11.67,37.14,21.13,27.0,-0.36,1.60,19.62,X40,ATH,PAINTS
44,KANSAINER,340.00,-67.24,58.0,H-SC,1.85,225900.0,-43767.0,80104.0,2.38,-16.23,35.46,13.47,138.0,-0.55,1.62,14.86,XY24,NTT,PAINTS
4,ANGELONE,3033.00,16.84,71.0,X-SC,8.78,207198.0,16192.0,41502.0,1.78,8.48,20.03,30.21,157.0,0.39,1.49,31.13,X40N,NTT,FINANCE
56,RELIANCE,1533.00,-12.91,57.0,X-LC,2.88,218135.0,7769.0,21006.0,1.75,3.69,9.63,13.68,37.0,0.37,1.56,20.83,XY25,NTT,REFINERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-14.25,45.0,M-LC,5.68,150641.0,-304.0,109757.0,-4.27,-0.20,72.86,72.51,53.0,-0.00,1.08,5.81,XR,NTT,IT
53,RAJOOENG,143.1,-44.70,36.0,H-SC,18.32,83710.0,-18790.0,59392.0,-1.83,-18.33,70.95,39.61,114.0,-0.32,0.60,2.23,AR,ATH,MISC
7,ATULAUTO,844.0,3640.54,50.0,M-SC,3.75,167994.0,-29138.0,118973.0,-1.82,-14.78,70.82,45.57,236.0,-0.24,1.20,19.10,XY24,NTT,AUTO
77,UJJIVANSFB,60.0,106.53,57.0,H-SC,12.27,130383.0,-12096.0,31618.0,-1.43,-8.49,24.25,13.70,163.0,-0.38,0.94,53.89,OX40N,NTT,BANKS
54,RECLTD,446.0,45.36,47.0,M-LC,6.23,204875.0,1705.0,40422.0,-1.32,0.84,19.73,20.74,55.0,0.04,1.47,6.43,XY25,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.69,149814.0,594.0,143582.0,-0.26,0.40,95.84,96.62,208.0,0.00,1.07,62.13,XY25,NTT,FINANCE
59,SAIL,228.00,42.74,48.0,M-MC,12.12,225065.0,103.0,167561.0,-0.52,0.05,74.45,74.53,192.0,0.00,1.61,32.31,XY24,BTT,STEEL
36,INDIAMART,4810.62,-55.76,38.0,H-SC,7.90,123750.0,414.0,131212.0,-0.90,0.34,106.03,106.72,119.0,0.00,0.89,23.79,AR,ATH,MISC
54,RECLTD,446.00,45.36,47.0,M-LC,6.23,204875.0,1705.0,40422.0,-1.32,0.84,19.73,20.74,55.0,0.04,1.47,6.43,XY25,NTT,FINANCE
85,WIPRO,420.00,-14.25,45.0,M-LC,5.68,150641.0,-304.0,109757.0,-4.27,-0.20,72.86,72.51,53.0,-0.00,1.08,5.81,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,106.53,57.0,H-SC,12.27,130383.0,-12096.0,31618.0,-1.43,-8.49,24.25,13.70,163.0,-0.38,0.94,53.89,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,97.65,52.0,M-SC,1.63,85124.0,-15653.0,15748.0,-0.66,-15.53,18.50,0.09,245.0,-0.99,0.61,11.83,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.24,58.0,H-SC,1.85,225900.0,-43767.0,80104.0,2.38,-16.23,35.46,13.47,138.0,-0.55,1.62,14.86,XY24,NTT,PAINTS
17,CERA,9475.0,-23.39,43.0,H-SC,2.32,139771.0,-36132.0,78160.0,0.63,-20.54,55.92,23.89,149.0,-0.46,1.00,20.71,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.55,44.0,H-MC,7.34,102002.0,-29833.0,72054.0,-0.65,-22.63,70.64,32.03,98.0,-0.41,0.73,14.25,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,45.36,47.0,M-LC,6.23,204875.0,1705.0,40422.0,-1.32,0.84,19.73,20.74,55.0,0.04,1.47,6.43,XY25,NTT,FINANCE
59,SAIL,228.00,42.74,48.0,M-MC,12.12,225065.0,103.0,167561.0,-0.52,0.05,74.45,74.53,192.0,0.00,1.61,32.31,XY24,BTT,STEEL
31,HINDZINC,730.22,35.55,65.0,M-LC,10.06,221002.0,15926.0,96644.0,0.27,7.77,43.73,54.89,52.0,0.16,1.59,33.73,X5K,ATH,METALS
78,UNIONBANK,163.00,-13.18,51.0,M-LC,8.76,156110.0,15270.0,29224.0,-1.09,10.84,18.72,31.59,66.0,0.52,1.12,40.20,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,56.0,H-MC,12.22,191343.0,11535.0,100876.0,-0.81,6.42,52.72,62.52,88.0,0.11,1.37,42.69,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,35.55,65.0,M-LC,10.06,221002.0,15926.0,96644.0,0.27,7.77,43.73,54.89,52.0,0.16,1.59,33.73,X5K,ATH,METALS
11,BANKINDIA,190.00,-25.47,56.0,H-MC,12.22,191343.0,11535.0,100876.0,-0.81,6.42,52.72,62.52,88.0,0.11,1.37,42.69,XR,NTT,BANKS
84,WHIRLPOOL,2270.00,-41.17,64.0,M-SC,4.30,99510.0,8012.0,70742.0,7.19,8.76,71.09,86.07,223.0,0.11,0.71,44.51,XR,NTT,DURABLES
85,WIPRO,420.00,-14.25,45.0,M-LC,5.68,150641.0,-304.0,109757.0,-4.27,-0.20,72.86,72.51,53.0,-0.00,1.08,5.81,XR,NTT,IT
36,INDIAMART,4810.62,-55.76,38.0,H-SC,7.90,123750.0,414.0,131212.0,-0.90,0.34,106.03,106.72,119.0,0.00,0.89,23.79,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,7.44,144601.0,-131922.0,241990.0,0.39,-47.71,167.35,39.81,54.0,-0.55,1.04,1.92,XY24,NTT,AUTO
51,QUESS,986.00,-47.66,34.0,X-SC,38.66,51804.0,-13202.0,164131.0,-0.76,-20.31,316.83,232.18,198.0,-0.08,0.37,1.02,XY24,NTT,MISC
28,HAVELLS,2069.16,-6.89,40.0,X-MC,2.13,230677.0,-17054.0,92109.0,0.20,-6.88,39.93,30.30,92.0,-0.19,1.65,5.49,X40,ATH,ELECTRICAL
12,BATAINDIA,2096.00,-36.44,41.0,X-SC,4.31,95110.0,-33560.0,78856.0,-0.16,-26.08,82.91,35.20,219.0,-0.43,0.68,9.16,X40,NTT,FOOTWEAR
55,RELAXO,1176.00,-39.78,42.0,X-SC,4.42,82062.0,-63098.0,139029.0,-1.07,-43.47,169.42,52.31,136.0,-0.45,0.59,9.84,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-3.25,44.0,H-LC,0.38,155495.0,-30600.0,78074.0,-0.29,-16.44,50.21,25.51,15.0,-0.39,1.12,14.59,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,2.14,61.0,X-MC,1.35,214350.0,22608.0,15155.0,0.95,11.79,7.07,19.69,99.0,1.49,1.54,19.39,XY25,NTT,AC
28,HAVELLS,2069.16,-6.89,40.0,X-MC,2.13,230677.0,-17054.0,92109.0,0.20,-6.88,39.93,30.30,92.0,-0.19,1.65,5.49,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-14.07,64.0,X-MC,2.32,147919.0,11786.0,19333.0,-0.56,8.66,13.07,22.86,91.0,0.61,1.06,22.36,X40,ATH,INSURANCE
41,ITC,452.00,-38.77,51.0,X-LC,2.88,198118.0,-2020.0,22902.0,1.31,-1.01,11.56,10.44,4.0,-0.09,1.42,4.50,X40,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,34.0,X-SC,38.66,51804.0,-13202.0,164131.0,-0.76,-20.31,316.83,232.18,198.0,-0.08,0.37,1.02,XY24,NTT,MISC
72,TATAMOTORS,1065.00,-54.49,14.0,X-LC,7.44,144601.0,-131922.0,241990.0,0.39,-47.71,167.35,39.81,54.0,-0.55,1.04,1.92,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-14.90,42.0,X-MC,6.04,179416.0,-21090.0,118235.0,-0.58,-10.52,65.90,48.45,90.0,-0.18,1.29,3.46,X40N,ATH,FINANCE
73,TCS,4389.97,-29.65,47.0,X-LC,12.62,280317.0,-65639.0,132338.0,0.38,-18.97,47.21,19.28,1.0,-0.50,2.01,3.63,X40,ATH,IT
2,ACC,3906.00,-37.64,52.0,X-MC,3.25,185900.0,-52051.0,204694.0,-0.06,-21.87,110.11,64.15,174.0,-0.25,1.33,4.28,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.66,34.0,X-SC,38.66,51804.0,-13202.0,164131.0,-0.76,-20.31,316.83,232.18,198.0,-0.08,0.37,1.02,XY24,NTT,MISC
55,RELAXO,1176.00,-39.78,42.0,X-SC,4.42,82062.0,-63098.0,139029.0,-1.07,-43.47,169.42,52.31,136.0,-0.45,0.59,9.84,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.44,41.0,X-SC,4.31,95110.0,-33560.0,78856.0,-0.16,-26.08,82.91,35.20,219.0,-0.43,0.68,9.16,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-5.83,48.0,X-SC,0.84,107173.0,5169.0,38754.0,-0.08,5.07,36.16,43.06,122.0,0.13,0.77,23.05,X40N,ATH,MISC
32,HONAUT,58357.33,-22.88,46.0,X-SC,2.52,108645.0,-19293.0,66426.0,0.35,-15.08,61.14,36.84,143.0,-0.29,0.78,11.23,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-29.65,47.0,X-LC,12.62,280317.0,-65639.0,132338.0,0.38,-18.97,47.21,19.28,1.0,-0.50,2.01,3.63,X40,ATH,IT
39,INFY,2275.00,-19.60,46.0,X-LC,7.54,313430.0,542.0,171133.0,-0.20,0.17,54.60,54.87,3.0,0.00,2.25,6.83,X40,BTT,IT
41,ITC,452.00,-38.77,51.0,X-LC,2.88,198118.0,-2020.0,22902.0,1.31,-1.01,11.56,10.44,4.0,-0.09,1.42,4.50,X40,NTT,FMCG
82,VBL,671.64,-17.93,55.0,X-LC,10.51,295394.0,-20448.0,133105.0,0.37,-6.47,45.06,35.67,5.0,-0.15,2.12,6.48,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,48.0,H-LC,7.09,248904.0,-12715.0,131919.0,-0.51,-4.86,53.00,45.57,7.0,-0.10,1.79,5.91,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6677.78,54.0,L-SC,6.23,74391.0,-19419.0,96039.0,-1.01,-20.70,129.10,81.67,269.0,-0.20,0.53,44.06,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,48.0,L-SC,29.13,80640.0,-32909.0,72963.0,0.60,-28.98,90.48,35.27,268.0,-0.45,0.58,97.37,XR,NTT,HOTELS
48,MASFIN,398.61,-20.51,42.0,H-SC,9.48,90150.0,-7830.0,29434.0,-0.78,-7.99,32.65,22.05,152.0,-0.27,0.65,31.04,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,106.53,57.0,H-SC,12.27,130383.0,-12096.0,31618.0,-1.43,-8.49,24.25,13.70,163.0,-0.38,0.94,53.89,OX40N,NTT,BANKS
68,SURYODAY,240.00,55.70,52.0,H-SC,10.51,143463.0,-35608.0,101342.0,-0.30,-19.88,70.64,36.71,135.0,-0.35,1.03,42.34,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,18.69,149814.0,594.0,143582.0,-0.26,0.40,95.84,96.62,208.0,0.00,1.07,62.13,XY25,NTT,FINANCE
4,ANGELONE,3033.00,16.84,71.0,X-SC,8.78,207198.0,16192.0,41502.0,1.78,8.48,20.03,30.21,157.0,0.39,1.49,31.13,X40N,NTT,FINANCE
31,HINDZINC,730.22,35.55,65.0,M-LC,10.06,221002.0,15926.0,96644.0,0.27,7.77,43.73,54.89,52.0,0.16,1.59,33.73,X5K,ATH,METALS
67,STARHEALTH,761.00,21.02,69.0,H-SC,9.13,269362.0,-11761.0,144620.0,0.96,-4.18,53.69,47.26,144.0,-0.08,1.93,44.32,XY24,NTT,INSURANCE
63,SHALBY,327.00,1174.32,54.0,M-SC,23.37,173354.0,-7789.0,53254.0,-1.03,-4.30,30.72,25.10,235.0,-0.15,1.24,38.93,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,44.28
2,50,75.94


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.29
LC    33.86
MC    21.82
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.55
X40      14.82
XY25     12.35
XR       10.87
X40N     10.25
AR        8.14
OX40N     7.62
X200      1.83
X5K       1.59
SR        1.12
MH        0.83
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.43
X-LC    22.84
X-MC    16.13
M-SC    12.73
M-LC     5.26
H-LC     4.74
X-SC     4.68
H-MC     3.78
M-MC     1.61
L-SC     1.45
L-LC     1.02
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.31,-2.84,36.73
IT,12.86,-17.27,80.56
FINANCE,12.21,-11.21,61.62
MISC,6.83,-20.68,87.55
BANKS,6.30,-13.67,73.47
PAINTS,5.71,-16.00,33.29
ELECTRICAL,5.34,-10.71,49.55
INSURANCE,3.94,-0.50,38.07
AUTO,2.83,-45.73,115.40


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3313121.0,22
XR,1393418.0,14
AR,1196141.0,9
X40,769232.0,10
X40N,599568.0,8
OX40N,583226.0,10
XY25,458902.0,8
SR,265253.0,2
X5K,96644.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3456640.0,25
M-SC,1262444.0,15
X-MC,1076815.0,11
X-LC,1052935.0,13
X-SC,528698.0,6
H-MC,351758.0,3
H-LC,280730.0,3
M-LC,276047.0,4
L-SC,258864.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1190626.0      6
           XR         833065.0      7
           AR         824587.0      5
M-SC       XY24       690723.0      6
X-MC       XY24       583304.0      4
X-LC       X40        475783.0      5
           XY24       308724.0      2
X-SC       X40N       285711.0      4
H-SC       OX40N      275071.0      4
           SR         265253.0      2
X-MC       X40        214593.0      4
H-LC       AR         209993.0      2
X-LC       X40N       195622.0      3
H-MC       XY24       178828.0      1
L-SC       XR         169002.0      2
M-MC       XY24       167561.0      1
X-SC       XY24       164131.0      1
M-SC       AR         161561.0      2
X-MC       XY25       160683.0      2
M-SC       OX40N      146239.0      4
           XY25       143582.0      1
           XR         120339.0      2
X-MC       X40N       118235.0      1
M-LC       XR         109757.0      1
H-MC       XR         100876.0      1
M-LC       X5K         96644.0      1
L-SC       OX40N       89862.0      1
X-SC       X40         78856.0      1
X-LC       XY25        72806.0      3
H-MC       OX40N       72054.0      1
H-LC       X200        70737.0      1
H-SC       MH          68038.0      1
L-MC       XR          60379.0      1
L-LC       XY25        41409.0      1
M-LC       XY25        40422.0      1
           XY24        29224.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
